In [3]:
!pip install -q langchain langchain-experimental langchain-openai langchain-google-genai langchain-community langchain-cohere pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [4]:
from DataManager import load_data
from RouteVarQuery import RouteVarQuery
from Stop import Stop
from StopQuery import StopQuery
from PathQuery import PathQuery
from GraphBuilder import buildGraph

In [5]:
route_vars, all_stops = load_data(
  'data/vars.json',
  'data/stops.json',
  'data/paths.json'
)
graph = buildGraph(route_vars, all_stops)
dists, trace = graph.dijkstra_all()

In [6]:
def bus_shortest_path(start_stop_id, end_stop_id):
    path = graph.shortest_path(start_stop_id, end_stop_id, dists, trace)
    time = sum([stop['time'] for stop in path if 'time' in stop])
    dist = sum([stop['dist'] for stop in path if 'dist' in stop])
    return path, time, dist

route_var_query = RouteVarQuery(list(route_vars.values()))
stop_query = StopQuery(list(all_stops.values()))
path_query = PathQuery([route_var.Path for route_var in route_vars.values()])

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_cohere import ChatCohere
from langchain_core.tools import tool
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.agents.agent import RunnableMultiActionAgent
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

COHERE_API_KEY = "..."
GOOGLE_API_KEY = "..."

In [11]:
# llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
llm = ChatCohere(model="command-r-plus", cohere_api_key=COHERE_API_KEY, temperature=0)

In [8]:
import re
import ast
from contextlib import redirect_stdout
from io import StringIO
from typing import Optional
from langchain_core.callbacks.manager import CallbackManagerForToolRun

class FixedPythonAstREPLTool(PythonAstREPLTool):
    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        io_buffer = StringIO()
        try:
            if self.sanitize_input:
                # Removes `, whitespace & python from start
                query = re.sub(r"^(\s|`)*(?i:python)?\s*", "", query)
                # Removes whitespace & ` from end
                query = re.sub(r"(\s|`)*$", "", query)
            print('Will execute this script:')
            print(query)
            print()
            tree = ast.parse(query)
            module = ast.Module(tree.body[:-1], type_ignores=[])
            with redirect_stdout(io_buffer):
                exec(ast.unparse(module), self.globals)
            module_end = ast.Module(tree.body[-1:], type_ignores=[])
            module_end_str = ast.unparse(module_end)
            try:
                with redirect_stdout(io_buffer):
                    ret = eval(module_end_str, self.globals)
                return io_buffer.getvalue() + (f'\n\nThe script returns:\n{ret}' if ret else '')
            except Exception:
                with redirect_stdout(io_buffer):
                    exec(module_end_str, self.globals)
                return io_buffer.getvalue()
        except Exception as e:
            return io_buffer.getvalue() + "\n\nThe script terminated because of:\n{}: {}".format(type(e).__name__, str(e))

tools = [FixedPythonAstREPLTool(globals={'stop_query': stop_query, 'bus_shortest_path': bus_shortest_path})]

In [9]:
system_message = """\
You are working with a Bus Network. The Bus Network consists of several Bus Stops.
Each Bus Stop has the following properties:
```
{
    "StopId": integer,
    "Code": string,
    "Name": string,
    "StopType": string,
    "Zone": string, // Zone is District name in Vietnamese (eg. "Quận 1")
    "Ward": string | None,
    "AddressNo": string,
    "Street": string,
    "SupportDisability": string,
    "Status": string,
    "Lng": float,
    "Lat": float,
    "Search": string,
    "Routes": string
}
```
Remember that Stop object only have these properties.

Here is an example Stop:
```
Stop(
  StopId=7276
  Code="Q1 190"
  Name="Tôn Thất Tùng"
  StopType="Trụ dừng"
  Zone="Quận 1"
  Ward="Phường Phạm Ngũ Lão"
  AddressNo="277-279-275Y"
  Street="Phạm Ngũ Lão"
  SupportDisability="Có"
  Status="Đang khai thác"
  Lng=106.690941
  Lat=10.76767
  Search="TTT 277-279-275Y PNL"
  Routes="03, 04, 102, 109, 140, 18, 19, 20, 28, 34, 36, 39, 46, 52, 65, 69, 72, 75, 86, 88, 93"
)

```

You have access to Python and can query the Stops by using the `stop_query` object. The `stop_query` supports a operation:
`stop_query.searchByAttribute(attribute_name: str, attribute_value: Any)`

For example:
`stop_query.searchByAttribute("Zone", "Quận 1")` to find all stop in District 1.

You can chain stop_query together for complex queries. For example:
`stop_query.searchByAttribute("Zone", "Quận 1").searchByAttribute("Ward", "3")` to find all stop in District 1, Ward 3.

You can access the list of Stops from a stop_query using `stop_query.Stops`. This is very useful for complex queries.
For example, if the user wants to get all stop that the route 150 go through in District 3:
```
filtered_stops = []
for stop in stop_query.Stops:
    if stop.Zone == "Quận 3" and "150" in stop.Routes:
        filtered_stops.append(stop)
        print(stop.Name)
```

You can safely assume that the `stop_query` object is already defined as a global variable.
You only have access to `stop_query` and built-in Python functions and modules.

Beside that, you also have the function `bus_shortest_path(start_stop_id: int, destination_stop_id: int) -> tuple[list[Stop], float, float]` to find the shortest path between 2 stops.
This function return 3 things: the path (list of PathPoint), total time in minutes and total distance in km.
If `path` is an empty list, there is no path from start stop to destination stop.

PathPoint structure is {
    "StopId": int,
    "RouteId": int, // bus route to take from this stop
    "RouteVarId": int, // bus route variant to take from this stop
    "time": float, // time to next stop
    "dist": float, // distance to next stop
}

For example, if the user asked about the path from stop 431 to stop 442, you can run:
```
path, time, distance = bus_shortest_path(431, 442)
print(f"Total time: {time} minutes, Total distance: {distance} km")

# Print the path
for point in path:
    stop_id = point["StopId"]
    stop = next(filter(lambda s: s.StopId == stop_id, stop_query.Stops))
    print(f"{stop.StopId}: {stop.Name}")
```

If you receive an error, try to rewrite the code and try again. Never give up!
"""

system_message = SystemMessage(content=system_message)
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [12]:
agent = RunnableMultiActionAgent(
    runnable=create_tool_calling_agent(llm, tools, prompt),
    input_keys_arg=["input"],
    return_keys_arg=["output"],
)

agent = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=False,
    max_iterations=15,
    early_stopping_method='force'
)

In [ ]:
agent.invoke("How to get from Chu Văn An to Ngã tư Phú Nhuận?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'start_stop = next(filter(lambda s: s.Name == "Chu Văn An", stop_query.Stops))\ndestination_stop = next(filter(lambda s: s.Name == "Ngã tư Phú Nhuận", stop_query.Stops))\n\npath, time, distance = bus_shortest_path(start_stop.StopId, destination_stop.StopId)\nprint(f"Total time: {time} minutes, Total distance: {distance} km")\n\n# Print the path\nfor point in path:\n    stop_id = point["StopId"]\n    stop = next(filter(lambda s: s.StopId == stop_id, stop_query.Stops))\n    print(f"{stop.StopId}: {stop.Name}")'}`


Will execute this script:
start_stop = next(filter(lambda s: s.Name == "Chu Văn An", stop_query.Stops))
destination_stop = next(filter(lambda s: s.Name == "Ngã tư Phú Nhuận", stop_query.Stops))

path, time, distance = bus_shortest_path(start_stop.StopId, destination_stop.StopId)
print(f"Total time: {time} minutes, Total distance: {distance} km")

# Print the path
for point in path:
    stop_id

{'input': 'How to get from Chu Văn An to Ngã tư Phú Nhuận?',
 'output': 'To get from Chu Văn An to Ngã tư Phú Nhuận, it will take around **27 minutes** and you will travel around **11.13 km**. Here is the path you will take:\n1. Chu Văn An\n2. Thuận Kiều Plaza\n3. Bệnh viện Chợ Rẫy\n4. Đại học Y Dược\n5. Bệnh viện Đại học Y Dược\n6. Công viên Văn Lang\n7. Trường Trần Khai Nguyên\n8. Trường Khiếm thị Nguyễn Đình Chiểu\n9. Hòa Hảo\n10. Đài tưởng niệm\n11. Ngã 7 Lý Thái Tổ\n12. Cao Thắng\n13. Cư Xá Đô Thành\n14. Bệnh viện Bình Dân\n15. Cách Mạng tháng 8\n16. Bệnh viện Mắt TPHCM\n17. Sở Khoa học và Công nghệ\n18. Phạm Ngọc Thạch\n19. Công Viên Lê Văn Tám\n20. Đinh Tiên Hoàng\n21. Cầu Bông\n22. Bệnh viện Bình Thạnh\n23. Lăng Ông Bà Chiểu\n24. UBND Quận Bình Thạnh\n25. Bệnh Viện Gia Định\n26. PCCC quận Bình Thạnh\n27. Bệnh viện Phước An\n28. Đại học Văn Hiến\n29. Ngã Tư Thích Quảng Đức\n30. Ngã tư Phan Xích Long\n31. Ngã tư Phú Nhuận'}

In [13]:
agent.invoke("Info about stop Cao Thắng")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'next(filter(lambda s: s.Name == "Cao Thắng", stop_query.Stops)).__dict__'}`


Will execute this script:
next(filter(lambda s: s.Name == "Cao Thắng", stop_query.Stops)).__dict__



The script returns:
{'_StopId': 92, '_Code': 'Q3 021', '_Name': 'Cao Thắng', '_StopType': 'Ô sơn', '_Zone': 'Quận 3', '_Ward': 'Phường 01', '_AddressNo': '503-507', '_Street': 'Điện Biên Phủ', '_SupportDisability': '', '_Status': 'Đang khai thác', '_Lng': 106.67738, '_Lat': 10.771164, '_Search': 'CT 503-507 DBP', '_Routes': '150, 91'}Here is some information about the Cao Thắng bus stop:
- **Stop ID:** 92
- **Code:** Q3 021
- **Zone:** Quận 3
- **Address:** 503-507 Điện Biên Phủ Street
- **Status:** Đang khai thác
- **Bus Routes:** 150 and 91

> Finished chain.


{'input': 'Info about stop Cao Thắng',
 'output': 'Here is some information about the Cao Thắng bus stop:\n- **Stop ID:** 92\n- **Code:** Q3 021\n- **Zone:** Quận 3\n- **Address:** 503-507 Điện Biên Phủ Street\n- **Status:** Đang khai thác\n- **Bus Routes:** 150 and 91'}